# Importing

In [7]:
import pandas as pd
import nltk
import spacy
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from sklearn.datasets import make_classification
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
import re
import pycld2 as cld2
from langdetect import detect
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score, classification_report 
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import RocCurveDisplay, plot_roc_curve
#from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adrianacuppuleri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df = pd.read_csv("data/WELFake_Dataset.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
df

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71576 non-null  object
 1   text    72095 non-null  object
 2   label   72134 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [5]:
df['label'].value_counts()

1    37106
0    35028
Name: label, dtype: int64

In [6]:
df[df['title'].isna() & df['text'].isna()]

,title,text,label


In [7]:
df[df['title'].isna() | df['text'].isna()]

,title,text,label
1,NaN,Did they post their votes for Hillary already?,1
43,NaN,True. Hillary needs a distraction and what bet...,1
162,NaN,All eyes on Electoral delegates. The People kn...,1
185,NaN,Cool,1
269,NaN,A leading US senator: US Supporting War in Syr...,1
...,...,...,...
71484,NaN,Another Arab supremacist masturbation fantasy....,1
71521,NaN,I'm sure they drastically changed accounting m...,1
71540,NaN,It's easy to imagine Obama or Kerry pissing hi...,1
71570,NaN,Ever since the powers to be assassinated JFK A...,1


In [8]:
df.dropna(axis=0, how='any', inplace=True)

In [9]:
df.reset_index(drop=True, inplace=True)
df

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
...,...,...,...
71532,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
71533,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
71534,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
71535,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


# Text preprocessing
## With NLTK

### Cleaning text

In [13]:
def clean(text):
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)           # sequences of white spaces
    text = re.sub(r'[^\x00-\x7F]+',' ', text)  # Removing all the non ASCII characters
    text = re.sub(r'\s+',' ', text)            # Replacing multiple Spaces with Single Space
    text = re.sub(r'\.{2,}', ' ', text)        # Replacing Two or more dots with one
    text = re.sub(r'[^\x00-\x7F]+',' ', text)  # Removing all the non ASCII characters
    text = re.sub(r'\W+',' ', text)            # Replace everything non-alpahnumeric with a space
    return text.strip()

In [14]:
df['text_clean'] = df['text'].map(clean)
df['title_clean'] = df['title'].map(clean)

In [15]:
df["empty_cell_text"] = df['text_clean'].str.contains(r'^\s*$', na=False)
df["empty_cell_title"] = df['title_clean'].str.contains(r'^\s*$', na=False)

In [16]:
df

,title,text,label,text_clean,title_clean,empty_cell_text,empty_cell_title
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...,False,False
1,NaN,Did they post their votes for Hillary already?,1,did they post their votes for hillary already,nan,False,False
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...,False,False
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...,False,False
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...,False,False
...,...,...,...,...,...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...,False,False
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...,False,False
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,migrants refuse to leave train at refugee camp...,False,False
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,mexico city reuters donald trump s combative s...,trump tussle gives unpopular mexican leader mu...,False,False


### Removing empty cells

In [17]:
df.drop(df.loc[df["empty_cell_text" or "empty_cell_title"]].index, inplace=True)
df

,title,text,label,text_clean,title_clean,empty_cell_text,empty_cell_title
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...,False,False
1,NaN,Did they post their votes for Hillary already?,1,did they post their votes for hillary already,nan,False,False
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...,False,False
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...,False,False
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...,False,False
...,...,...,...,...,...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...,False,False
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...,False,False
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,migrants refuse to leave train at refugee camp...,False,False
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,mexico city reuters donald trump s combative s...,trump tussle gives unpopular mexican leader mu...,False,False


In [18]:
df.reset_index(drop=True, inplace=True)
df.drop(columns=["empty_cell_text", "empty_cell_title"], inplace=True)

In [19]:
df

,title,text,label,text_clean,title_clean
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...
1,NaN,Did they post their votes for Hillary already?,1,did they post their votes for hillary already,nan
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...
...,...,...,...,...,...
71372,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...
71373,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...
71374,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,migrants refuse to leave train at refugee camp...
71375,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,mexico city reuters donald trump s combative s...,trump tussle gives unpopular mexican leader mu...


In [17]:
#df.to_csv("data/df_cleaned.csv")

### Language detection

In [18]:
def detect_lang(text):
    _, _, _, detected_language = cld2.detect(text, returnVectors=True)
    return str(detected_language)

In [19]:
df['text_lang'] = df['text_clean'].map(detect_lang)
df['title_lang'] = df['title_clean'].map(detect_lang)

In [20]:
df['text_lang'].astype(str)
df['title_lang'].astype(str)

0        ((0, 126, 'ENGLISH', 'en'),)
1        ((0, 132, 'ENGLISH', 'en'),)
2        ((0, 103, 'ENGLISH', 'en'),)
3         ((0, 90, 'ENGLISH', 'en'),)
4         ((0, 77, 'ENGLISH', 'en'),)
                     ...             
70776     ((0, 64, 'ENGLISH', 'en'),)
70777     ((0, 77, 'ENGLISH', 'en'),)
70778     ((0, 57, 'ENGLISH', 'en'),)
70779     ((0, 67, 'ENGLISH', 'en'),)
70780     ((0, 52, 'Unknown', 'un'),)
Name: title_lang, Length: 70781, dtype: object

In [21]:
df['text_lang'] = ~df["text_lang"].str.contains('ENGLISH|Unknown', regex=True)

In [22]:
df['title_lang'] = ~df["title_lang"].str.contains('ENGLISH|Unknown')

In [23]:
df

,title,text,label,text_clean,title_clean,text_lang,title_lang
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...,False,False
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...,False,False
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...,False,False
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...,False,False
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,all we can say on this one is it s about time ...,about time christian group sues amazon and spl...,False,False
...,...,...,...,...,...,...,...
70776,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...,False,False
70777,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...,False,False
70778,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,migrants refuse to leave train at refugee camp...,False,False
70779,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,mexico city reuters donald trump s combative s...,trump tussle gives unpopular mexican leader mu...,False,False


In [24]:
df.drop(df.loc[df["text_lang" or "title_lang"]].index, inplace=True)

In [25]:
df

,title,text,label,text_clean,title_clean,text_lang,title_lang
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...,False,False
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...,False,False
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...,False,False
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...,False,False
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,all we can say on this one is it s about time ...,about time christian group sues amazon and spl...,False,False
...,...,...,...,...,...,...,...
70776,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...,False,False
70777,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...,False,False
70778,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,migrants refuse to leave train at refugee camp...,False,False
70779,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,mexico city reuters donald trump s combative s...,trump tussle gives unpopular mexican leader mu...,False,False


In [26]:
df.reset_index(drop=True, inplace=True)

### Tokenizing

In [27]:
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
df['token_title'] = df.apply(lambda row: nltk.word_tokenize(row['title_clean']), axis=1)

In [28]:
df['token_text'] = df.apply(lambda row: nltk.word_tokenize(row['text_clean']), axis=1)

In [29]:
df.drop(columns=["text_lang", "title_lang"], inplace=True)

In [30]:
df.to_csv("data/df_token.csv")

In [31]:
#df = pd.read_csv("df_token.csv")
#df.drop(columns=["Unnamed: 0"], inplace=True)
#df

In [32]:
df.token_title[0]

['law',
 'enforcement',
 'on',
 'high',
 'alert',
 'following',
 'threats',
 'against',
 'cops',
 'and',
 'whites',
 'on',
 '9',
 '11by',
 'blacklivesmatter',
 'and',
 'fyf911',
 'terrorists',
 'video']

### POS tagging

In [33]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [34]:
df['tag_title'] = df.apply(lambda row: nltk.pos_tag(row['token_title']), axis=1)

In [ ]:
#df['tag_text'] = df.apply(lambda row: nltk.pos_tag(row['token_text']), axis=1)

In [36]:
df.tag_title[0]

[('law', 'NN'),
 ('enforcement', 'NN'),
 ('on', 'IN'),
 ('high', 'JJ'),
 ('alert', 'NN'),
 ('following', 'VBG'),
 ('threats', 'NNS'),
 ('against', 'IN'),
 ('cops', 'NNS'),
 ('and', 'CC'),
 ('whites', 'NNS'),
 ('on', 'IN'),
 ('9', 'CD'),
 ('11by', 'CD'),
 ('blacklivesmatter', 'NN'),
 ('and', 'CC'),
 ('fyf911', 'NN'),
 ('terrorists', 'NNS'),
 ('video', 'VBP')]

### Lemmatizing tagged words

In [37]:
lem = WordNetLemmatizer()

In [38]:
def lemmatize(words):
    lemmatized_words = [lem.lemmatize(word) for word in words]
    return lemmatized_words

In [39]:
df['lem_title'] = df.apply(lambda row: lemmatize(row['token_title']), axis=1)

In [40]:
df['lem_text'] = df.apply(lambda row: lemmatize(row['token_text']), axis=1)

In [41]:
df

,title,text,label,text_clean,title_clean,token_title,token_text,tag_title,lem_title,lem_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...,"[law, enforcement, on, high, alert, following,...","[no, comment, is, expected, from, barack, obam...","[(law, NN), (enforcement, NN), (on, IN), (high...","[law, enforcement, on, high, alert, following,...","[no, comment, is, expected, from, barack, obam..."
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...,"[unbelievable, obama, s, attorney, general, sa...","[now, most, of, the, demonstrators, gathered, ...","[(unbelievable, JJ), (obama, NN), (s, NN), (at...","[unbelievable, obama, s, attorney, general, sa...","[now, most, of, the, demonstrator, gathered, l..."
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...,"[bobby, jindal, raised, hindu, uses, story, of...","[a, dozen, politically, active, pastors, came,...","[(bobby, NN), (jindal, NN), (raised, VBD), (hi...","[bobby, jindal, raised, hindu, us, story, of, ...","[a, dozen, politically, active, pastor, came, ..."
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...,"[satan, 2, russia, unvelis, an, image, of, its...","[the, rs, 28, sarmat, missile, dubbed, satan, ...","[(satan, JJ), (2, CD), (russia, NN), (unvelis,...","[satan, 2, russia, unvelis, an, image, of, it,...","[the, r, 28, sarmat, missile, dubbed, satan, 2..."
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,all we can say on this one is it s about time ...,about time christian group sues amazon and spl...,"[about, time, christian, group, sues, amazon, ...","[all, we, can, say, on, this, one, is, it, s, ...","[(about, IN), (time, NN), (christian, JJ), (gr...","[about, time, christian, group, sue, amazon, a...","[all, we, can, say, on, this, one, is, it, s, ..."
...,...,...,...,...,...,...,...,...,...,...
70680,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...,"[russians, steal, research, on, trump, in, hac...","[washington, reuters, hackers, believed, to, b...","[(russians, NNS), (steal, VBP), (research, NN)...","[russian, steal, research, on, trump, in, hack...","[washington, reuters, hacker, believed, to, be..."
70681,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...,"[watch, giuliani, demands, that, democrats, ap...","[you, know, because, in, fantasyland, republic...","[(watch, NN), (giuliani, NN), (demands, VBZ), ...","[watch, giuliani, demand, that, democrat, apol...","[you, know, because, in, fantasyland, republic..."
70682,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...,migrants refuse to leave train at refugee camp...,"[migrants, refuse, to, leave, train, at, refug...","[migrants, refuse, to, leave, train, at, refug...","[(migrants, NNS), (refuse, VBP), (to, TO), (le...","[migrant, refuse, to, leave, train, at, refuge...","[migrant, refuse, to, leave, train, at, refuge..."
70683,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,mexico city reuters donal

In [42]:
df.to_csv("data/df_lemmatized.csv")

for word, tag in enumerate(df['tag_title']):
         wntag = tag[0][0][0].lower()
         wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
         lemma = lem.lemmatize(word, wntag) if wntag else word
         print (lemma)

### Stopwords

In [43]:
stop_words = list(stopwords.words('english')) 
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [44]:
for i in range(len(stop_words)):
    stop_words[i] = re.sub(r"\s*'\s*\w*","",stop_words[i])

In [45]:
df["stop_title"] = df["lem_title"].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))

In [46]:
df["stop_text"] = df["lem_text"].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))

In [47]:
df

,title,text,label,text_clean,title_clean,token_title,token_text,tag_title,lem_title,lem_text,stop_title,stop_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...,"[law, enforcement, on, high, alert, following,...","[no, comment, is, expected, from, barack, obam...","[(law, NN), (enforcement, NN), (on, IN), (high...","[law, enforcement, on, high, alert, following,...","[no, comment, is, expected, from, barack, obam...",law enforcement high alert following threat co...,comment expected barack obama member fyf911 fu...
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...,"[unbelievable, obama, s, attorney, general, sa...","[now, most, of, the, demonstrators, gathered, ...","[(unbelievable, JJ), (obama, NN), (s, NN), (at...","[unbelievable, obama, s, attorney, general, sa...","[now, most, of, the, demonstrator, gathered, l...",unbelievable obama attorney general say charlo...,demonstrator gathered last night exercising co...
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...,"[bobby, jindal, raised, hindu, uses, story, of...","[a, dozen, politically, active, pastors, came,...","[(bobby, NN), (jindal, NN), (raised, VBD), (hi...","[bobby, jindal, raised, hindu, us, story, of, ...","[a, dozen, politically, active, pastor, came, ...",bobby jindal raised hindu us story christian c...,dozen politically active pastor came private d...
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...,"[satan, 2, russia, unvelis, an, image, of, its...","[the, rs, 28, sarmat, missile, dubbed, satan, ...","[(satan, JJ), (2, CD), (russia, NN), (unvelis,...","[satan, 2, russia, unvelis, an, image, of, it,...","[the, r, 28, sarmat, missile, dubbed, satan, 2...",satan 2 russia unvelis image terrifying new su...,r 28 sarmat missile dubbed satan 2 replace 18 ...
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,all we can say on this one is it s about time ...,about time christian group sues amazon and spl...,"[about, time, christian, group, sues, amazon, ...","[all, we, can, say, on, this, one, is, it, s, ...","[(about, IN), (time, NN), (christian, JJ), (gr...","[about, time, christian, group, sue, amazon, a...","[all, we, can, say, on, this, one, is, it, s, ...",time christian group sue amazon splc designati...,say one time someone sued southern poverty law...
...,...,...,...,...,...,...,...,...,...,...,...,...
70680,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...,"[russians, steal, research, on, trump, in, hac...","[washington, reuters, hackers, believed, to, b...","[(russians, NNS), (steal, VBP), (research, NN)...","[russian, steal, research, on, trump, in, hack...","[washington, reuters, hacker, believed, to, be...",russian steal research trump hack u democratic...,washington reuters hacker believed working rus...
70681,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...,"[watch, giuliani, demands, that, democrats, ap...","[you, know, because, in, fantasyland, republic...","[(watch, NN), (giuliani, NN), (demands, VBZ), ...","[watch, giuliani, demand, that, democrat, apol...","[you, know, because, in

In [48]:
df.to_csv("data/df_stopwords.csv")

### BOW with countvec [ignore]

In [49]:
def create_vectorizer(sentences):
    vectorizer = CountVectorizer(max_features=100)
    X = vectorizer.fit_transform(sentences)
    return (vectorizer, X)

In [50]:
(vectorizer, X) = create_vectorizer(df.stop_title)

In [51]:
print(vectorizer.get_feature_names())

['2016', 'america', 'american', 'anti', 'attack', 'back', 'ban', 'bill', 'black', 'border', 'breaking', 'breitbart', 'call', 'campaign', 'chief', 'china', 'clinton', 'cnn', 'comment', 'congress', 'could', 'court', 'day', 'deal', 'democrat', 'donald', 'election', 'email', 'eu', 'fbi', 'first', 'former', 'get', 'give', 'go', 'gop', 'government', 'gun', 'ha', 'hillary', 'house', 'iran', 'korea', 'law', 'leader', 'life', 'like', 'make', 'man', 'may', 'medium', 'million', 'muslim', 'new', 'news', 'north', 'obama', 'official', 'one', 'party', 'people', 'plan', 'police', 'president', 'putin', 'report', 'republican', 'right', 'russia', 'russian', 'sander', 'say', 'senate', 'senator', 'show', 'speech', 'state', 'supporter', 'syria', 'take', 'talk', 'tax', 'tell', 'time', 'top', 'trump', 'tweet', 'video', 'vote', 'voter', 'wa', 'want', 'war', 'watch', 'white', 'win', 'woman', 'world', 'year', 'york']


In [52]:
print(X)

  (0, 43)	1
  (0, 94)	1
  (0, 87)	1
  (1, 87)	1
  (1, 56)	1
  (1, 71)	1
  (1, 76)	1
  (1, 55)	1
  (2, 0)	1
  (3, 68)	1
  (3, 53)	1
  (3, 97)	1
  (3, 79)	1
  (4, 83)	1
  (6, 87)	1
  (6, 6)	1
  (6, 74)	1
  (6, 2)	1
  (6, 46)	1
  (6, 1)	1
  (8, 35)	1
  (8, 73)	1
  (8, 67)	1
  (9, 49)	1
  (9, 28)	2
  :	:
  (70676, 53)	1
  (70676, 83)	1
  (70676, 85)	1
  (70676, 99)	1
  (70677, 43)	1
  (70677, 37)	1
  (70678, 74)	1
  (70678, 16)	2
  (70678, 27)	1
  (70679, 87)	1
  (70679, 83)	1
  (70679, 75)	1
  (70679, 5)	1
  (70680, 85)	1
  (70680, 59)	1
  (70680, 69)	1
  (70681, 85)	1
  (70681, 93)	1
  (70681, 24)	1
  (70683, 85)	1
  (70683, 33)	1
  (70683, 44)	1
  (70684, 16)	1
  (70684, 63)	1
  (70684, 39)	1


In [53]:
denseX = X.todense()

In [54]:
denseX[0]

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [55]:
df

,title,text,label,text_clean,title_clean,token_title,token_text,tag_title,lem_title,lem_text,stop_title,stop_text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,no comment is expected from barack obama membe...,law enforcement on high alert following threat...,"[law, enforcement, on, high, alert, following,...","[no, comment, is, expected, from, barack, obam...","[(law, NN), (enforcement, NN), (on, IN), (high...","[law, enforcement, on, high, alert, following,...","[no, comment, is, expected, from, barack, obam...",law enforcement high alert following threat co...,comment expected barack obama member fyf911 fu...
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,now most of the demonstrators gathered last ni...,unbelievable obama s attorney general says mos...,"[unbelievable, obama, s, attorney, general, sa...","[now, most, of, the, demonstrators, gathered, ...","[(unbelievable, JJ), (obama, NN), (s, NN), (at...","[unbelievable, obama, s, attorney, general, sa...","[now, most, of, the, demonstrator, gathered, l...",unbelievable obama attorney general say charlo...,demonstrator gathered last night exercising co...
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,a dozen politically active pastors came here f...,bobby jindal raised hindu uses story of christ...,"[bobby, jindal, raised, hindu, uses, story, of...","[a, dozen, politically, active, pastors, came,...","[(bobby, NN), (jindal, NN), (raised, VBD), (hi...","[bobby, jindal, raised, hindu, us, story, of, ...","[a, dozen, politically, active, pastor, came, ...",bobby jindal raised hindu us story christian c...,dozen politically active pastor came private d...
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,the rs 28 sarmat missile dubbed satan 2 will r...,satan 2 russia unvelis an image of its terrify...,"[satan, 2, russia, unvelis, an, image, of, its...","[the, rs, 28, sarmat, missile, dubbed, satan, ...","[(satan, JJ), (2, CD), (russia, NN), (unvelis,...","[satan, 2, russia, unvelis, an, image, of, it,...","[the, r, 28, sarmat, missile, dubbed, satan, 2...",satan 2 russia unvelis image terrifying new su...,r 28 sarmat missile dubbed satan 2 replace 18 ...
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,all we can say on this one is it s about time ...,about time christian group sues amazon and spl...,"[about, time, christian, group, sues, amazon, ...","[all, we, can, say, on, this, one, is, it, s, ...","[(about, IN), (time, NN), (christian, JJ), (gr...","[about, time, christian, group, sue, amazon, a...","[all, we, can, say, on, this, one, is, it, s, ...",time christian group sue amazon splc designati...,say one time someone sued southern poverty law...
...,...,...,...,...,...,...,...,...,...,...,...,...
70680,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,washington reuters hackers believed to be work...,russians steal research on trump in hack of u ...,"[russians, steal, research, on, trump, in, hac...","[washington, reuters, hackers, believed, to, b...","[(russians, NNS), (steal, VBP), (research, NN)...","[russian, steal, research, on, trump, in, hack...","[washington, reuters, hacker, believed, to, be...",russian steal research trump hack u democratic...,washington reuters hacker believed working rus...
70681,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,you know because in fantasyland republicans ne...,watch giuliani demands that democrats apologiz...,"[watch, giuliani, demands, that, democrats, ap...","[you, know, because, in, fantasyland, republic...","[(watch, NN), (giuliani, NN), (demands, VBZ), ...","[watch, giuliani, demand, that, democrat, apol...","[you, know, because, in

## Modeling

### TF-IDF 

In [56]:
# on titles

X_train, X_test, Y_train, Y_test = train_test_split(df['stop_title'],
df['label'],
test_size=0.2,
random_state=42,
stratify=df['label'])

In [57]:
print ('Size of Training Data ', X_train.shape[0])
print ('Size of Test Data ', X_test.shape[0])
print ('Distribution of classes in Training Data :')
print ('Fake item ', str(sum(Y_train == 1)/ len(Y_train) * 100.0))
print ('Real item ', str(sum(Y_train == 0)/ len(Y_train) * 100.0))
print ('Distribution of classes in Testing Data :')
print ('Fake item ', str(sum(Y_test == 1)/ len(Y_test) * 100.0))
print ('Real item ', str(sum(Y_test == 0)/ len(Y_test) * 100.0))

Size of Training Data  56548
Size of Test Data  14137
Distribution of classes in Training Data :
Fake item  50.44563910306289
Real item  49.55436089693712
Distribution of classes in Testing Data :
Fake item  50.44917592134116
Real item  49.55082407865884


In [58]:
tfidf = TfidfVectorizer(max_features = 20000, ngram_range=(1,2))
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [59]:
SVC = LinearSVC(random_state=42, tol=1e-5)
SVC.fit(X_train_tf, Y_train)

LinearSVC(random_state=42, tol=1e-05)

In [60]:
Y_pred = SVC.predict(X_test_tf)
print ('Accuracy Score - ', accuracy_score(Y_test, Y_pred))
print ('ROC-AUC Score - ', roc_auc_score(Y_test, Y_pred))

Accuracy Score -  0.90981113390394
ROC-AUC Score -  0.9097383969386501


In [61]:
# on text

X_train, X_test, Y_train, Y_test = train_test_split(df['stop_text'],
df['label'],
test_size=0.2,
random_state=42,
stratify=df['label'])

In [62]:
print ('Size of Training Data ', X_train.shape[0])
print ('Size of Test Data ', X_test.shape[0])
print ('Distribution of classes in Training Data :')
print ('Fake item ', str(sum(Y_train == 1)/ len(Y_train) * 100.0))
print ('Real item ', str(sum(Y_train == 0)/ len(Y_train) * 100.0))
print ('Distribution of classes in Testing Data :')
print ('Fake item ', str(sum(Y_test == 1)/ len(Y_test) * 100.0))
print ('Real item ', str(sum(Y_test == 0)/ len(Y_test) * 100.0))

Size of Training Data  56548
Size of Test Data  14137
Distribution of classes in Training Data :
Fake item  50.44563910306289
Real item  49.55436089693712
Distribution of classes in Testing Data :
Fake item  50.44917592134116
Real item  49.55082407865884


In [63]:
tfidf = TfidfVectorizer(analyzer = 'word', max_features = 20000, ngram_range=(1,2))
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [64]:
SVC = LinearSVC(random_state=42, tol=1e-5)
SVC.fit(X_train_tf, Y_train)

LinearSVC(random_state=42, tol=1e-05)

In [65]:
Y_pred = SVC.predict(X_test_tf)
print ('Accuracy Score - ', accuracy_score(Y_test, Y_pred))
print ('ROC-AUC Score - ', roc_auc_score(Y_test, Y_pred))

Accuracy Score -  0.9654806536040178
ROC-AUC Score -  0.9654041480666602


In [66]:
tfidf.get_feature_names()

['00',
 '00 pm',
 '000',
 '000 000',
 '000 american',
 '000 email',
 '000 job',
 '000 mile',
 '000 new',
 '000 people',
 '000 per',
 '000 refugee',
 '000 rohingya',
 '000 syrian',
 '000 troop',
 '000 vote',
 '000 year',
 '01',
 '02',
 '03',
 '04',
 '05',
 '07',
 '08',
 '09',
 '10',
 '10 000',
 '10 2016',
 '10 2017',
 '10 billion',
 '10 day',
 '10 million',
 '10 minute',
 '10 month',
 '10 people',
 '10 percent',
 '10 point',
 '10 time',
 '10 year',
 '100',
 '100 000',
 '100 billion',
 '100 day',
 '100 million',
 '100 people',
 '100 percent',
 '100 year',
 '1000',
 '101',
 '10th',
 '11',
 '11 16',
 '11 2001',
 '11 2016',
 '11 2017',
 '11 attack',
 '11 day',
 '11 million',
 '11 percent',
 '11 year',
 '110',
 '115',
 '11th',
 '12',
 '12 000',
 '12 2016',
 '12 2017',
 '12 hour',
 '12 million',
 '12 month',
 '12 percent',
 '12 year',
 '120',
 '120 day',
 '125',
 '12th',
 '13',
 '13 2016',
 '13 2017',
 '13 million',
 '13 percent',
 '13 year',
 '130',
 '135',
 '13th',
 '14',
 '14 2016',
 '14 2

## Pretrained models and Transformers

In [67]:
!pip install transformers

In [2]:
import torch
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

In [71]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.3964, 0.8719, 0.6573],
        [0.7016, 0.5787, 0.2709],
        [0.7410, 0.1740, 0.7243],
        [0.2654, 0.0484, 0.3523],
        [0.5437, 0.2272, 0.4226]])


In [72]:
config = BertConfig.from_pretrained('bert-base-uncased',finetuning_task='binary')

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
def get_tokens(text, tokenizer, max_seq_length, add_special_tokens=True): 
    input_ids = tokenizer.encode(text, 
                                 add_special_tokens=add_special_tokens, 
                                 max_length=max_seq_length, 
                                 pad_to_max_length=True) 
    attention_mask = [int(id > 0) for id in input_ids] 
    assert len(input_ids) == max_seq_length 
    assert len(attention_mask) == max_seq_length 
    return (input_ids, attention_mask) 
text = "Here is the sentence I want embeddings for." 
input_ids, attention_mask = get_tokens(text, 
                                       tokenizer, 
                                       max_seq_length=30, 
                                       add_special_tokens = True) 
input_tokens = tokenizer.convert_ids_to_tokens(input_ids) 
print (text) 
print (input_tokens) 
print (input_ids) 
print (attention_mask)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Here is the sentence I want embeddings for.
['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2300: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
#XY train_test split + tokenize X_train and X_test
X_train, X_test, Y_train, Y_test = train_test_split(df['title_clean'],
                                                    df['label'], 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=df['label'])

X_train_tokens = X_train.apply(get_tokens, args=(tokenizer, 50)) 
X_test_tokens = X_test.apply(get_tokens, args=(tokenizer, 50))

/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2300: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
X_train_tokens.shape

(57101,)

In [26]:
Y_train.shape

(57101,)

In [27]:
#creation of 3 tensors:tokens, input masks and target labels
from torch.utils.data import TensorDataset 
input_ids_train = torch.tensor( 
    [features[0] for features in X_train_tokens.values], dtype=torch.long) 
input_mask_train = torch.tensor( 
    [features[1] for features in X_train_tokens.values], dtype=torch.long) 
label_ids_train = torch.tensor(Y_train.values, dtype=torch.long) 

print (input_ids_train.shape) 
print (input_mask_train.shape) 
print (label_ids_train.shape)

torch.Size([57101, 50])
torch.Size([57101, 50])
torch.Size([57101])


In [28]:
input_ids_train[1]

tensor([  101,  2490,  2005,  7987, 10288,  4183,  2012,  2274,  3204,  2152,
         2044,  3720,  2753, 13330,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [29]:
#combine tensors into a tensordataset
train_dataset = TensorDataset(input_ids_train,input_mask_train,label_ids_train)